In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
df = pd.read_csv('dataset/train_data.csv')

In [335]:
import matplotlib.pyplot as plt
import seaborn as sns

* Removing null values

In [400]:
df = df.dropna(how='any')
df = df.reset_index(drop=True)

* Converting time columns into pandas datetime

In [406]:
df['od_start_time'] = pd.to_datetime(df['od_start_time'])
df['od_end_time'] = pd.to_datetime(df['od_end_time'])
df['od_end_time']

0       2018-09-15 06:33:35.824186
1       2018-09-22 19:17:22.242888
2       2018-10-03 20:49:39.290326
3       2018-10-02 02:57:37.791139
4       2018-09-17 06:31:13.712576
                   ...            
39252   2018-09-13 09:30:55.753332
39253   2018-09-25 15:16:26.240232
39254   2018-09-13 19:19:53.803795
39255   2018-09-26 09:41:50.548373
39256   2018-09-15 08:08:11.723169
Name: od_end_time, Length: 39257, dtype: datetime64[ns]

In [338]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39257 entries, 0 to 39256
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   UID                             39257 non-null  object        
 1   trip_creation_time              39257 non-null  object        
 2   route_schedule_uuid             39257 non-null  object        
 3   route_type                      39257 non-null  object        
 4   trip_uuid                       39257 non-null  object        
 5   source_center                   39257 non-null  object        
 6   source_name                     39257 non-null  object        
 7   destination_center              39257 non-null  object        
 8   destination_name                39257 non-null  object        
 9   od_start_time                   39257 non-null  datetime64[ns]
 10  od_end_time                     39257 non-null  datetime64[ns]
 11  st

* Grouping by sub-journey in the trip

In [13]:
df['segment_key'] = df['trip_uuid'] + df['source_center'] + df['destination_center']

segment_cols = ['segment_actual_time', 'segment_osrm_distance', 'segment_osrm_time']

for col in segment_cols:
    df[col + '_sum'] = df.groupby('segment_key')[col].cumsum()
    
df[[col + '_sum' for col in segment_cols]]

,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum
0,77.988686,28.475783,20.793472
1,68.517914,34.612259,18.787231
2,NaN,NaN,NaN
3,101.876844,27.794019,19.588977
4,46.571301,26.971844,22.671495
...,...,...,...
72153,135.134614,116.816936,88.413572
72154,35.976514,15.139818,13.607794
72155,149.424478,84.944279,78.410963
72156,527.513633,160.142062,123.972095


In [403]:
df = df.dropna(how='any')
df

,UID,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,...,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time,segment_key,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum
0,lWYYrbNEdf,2018-09-15 02:23:18.656742,thanos::sroute:9d8ed3f7-a8af-4393-ad7a-c57a720...,FTL,trip-153697819865649878,IND832109AAB,Jamshedpur_Central_I_3 (Jharkhand),IND834002AAB,Ranchi_Hub (Jharkhand),2018-09-15 02:23:18.656742,...,2.782826,77.988686,20.793472,28.475783,3.292773,100.0,trip-153697819865649878IND832109AABIND834002AAB,77.988686,28.475783,20.793472
1,RrJUneuYNj,2018-09-21 05:11:44.833176,thanos::sroute:96a80600-40e1-436b-9161-fa68f9e...,FTL,trip-153750670483274503,IND842001AAA,Muzaffrpur_Bbganj_I (Bihar),IND854326AAB,Purnia_Central_H_2 (Bihar),2018-09-22 10:08:32.593022,...,2.946647,101.876844,19.588977,27.794019,4.265799,300.0,trip-153750670483274503IND842001AAAIND854326AAB,101.876844,27.794019,19.588977
2,aCQiP8akOa,2018-10-01 18:06:41.031369,thanos::sroute:0456b740-1dad-4929-bbe0-87d8843...,FTL,trip-153841720103113533,IND000000ACB,Gurgaon_Bilaspur_HB (Haryana),IND562132AAA,Bangalore_Nelmngla_H (Karnataka),2018-10-01 18:06:41.031369,...,2.513150,78.860571,35.429543,44.287435,2.661004,2489.0,trip-153841720103113533IND000000ACBIND562132AAA,78.860571,44.287435,35.429543
3,heWso9rqMM,2018-09-27 14:41:36.457278,thanos::sroute:3592c86e-c3d1-429b-917a-ebe9051...,FTL,trip-153805929645698249,IND712311AAA,Kolkata_Dankuni_HB (West Bengal),IND421302AAG,Bhiwandi_Mankoli_HB (Maharashtra),2018-09-29 18:54:31.061435,...,2.303103,78.117604,18.785432,30.457445,5.088583,1740.0,trip-153805929645698249IND712311AAAIND421302AAG,78.117604,30.457445,18.785432
4,d90XRsN4tX,2018-09-17 00:19:58.942475,thanos::sroute:9e13f8b8-837d-4541-8c0e-4475cfa...,Carting,trip-153714359894207097,IND614620AAA,Manamelkudi_TmpleSrt_D (Tamil Nadu),IND630561AAA,Sivaganga_Wardno3_D (Tamil Nadu),2018-09-17 03:45:22.278671,...,2.080589,15.308492,7.539141,10.093140,2.398321,80.0,trip-153714359894207097IND614620AAAIND630561AAA,15.308492,10.093140,7.539141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39252,LJvQJx1bJr,2018-09-13 01:54:33.375941,thanos::sroute:8a7e68b3-2a05-489f-85a1-5ccfbf6...,FTL,trip-153680367337554523,IND723130AAA,Hura_Central_D_1 (West Bengal),IND723101AAA,Purulia_GariDPP_D (West Bengal),2018-09-13 08:25:53.926550,...,2.091150,20.555236,18.044246,5.825889,2.917852,38.0,trip-153680367337554523IND723130AAAIND723101AAA,72.250437,33.754085,43.194972
39253,ywPz45CyQF,2018-09-24 03:17:20.331183,thanos::sroute:64d4c6c9-ffb9-4794-b9f1-05f064c...,FTL,trip-153775904033094104,IND110037AAM,Delhi_Airport_H (Delhi),IND421302AAG,Bhiwandi_Mankoli_HB (Maharashtra),2018-09-24 03:17:20.331183,...,2.873643,32.540267,19.519038,30.021599,2.194396,1723.0,trip-153775904033094104IND110037AAMIND421302AAG,932.134546,435.899237,347.770901
39254,hNwyLGzfvF,2018-09-13 09:40:42.505520,thanos::sroute:a89a51a1-073b-4e72-871e-56b911b...,Carting,trip-153683164250530836,IND000000AEL,Delhi_PunjabiB_L (Delhi),IND000000ACB,Gurgaon_Bilaspur_HB (Haryana),2018-09-13 09:40:42.505520,...,2.247334,16.260635,16.211918,17.894237,1.958930,56.0,trip-153683164250530836IND000000AELIND000000ACB,57.837635,46.656265,42.803766
39255,zNbaDEtUrG,2018-09-26 04:02:50.201585,thanos::sroute:1c165ebe-bad3-49b8-9756-087ec45...,FTL,trip-153793457020132273,IND524132AAA,Venktagiri_TirupthiRd_D (Andhra Pradesh),IND524101AAB,Gudur_Bngisheb_D (Andhra Pradesh),2018-09-26 07:54:50.012493,...,2.075224,48.190696,29.608287,32.399818,2.042262,35.0,trip-153793457020132273IND524132AAAIND524101AAB,48.190696,32.399818,29.608287


* aggregating at sub-jouney lebel

In [10]:
create_segment_dict = {
    
    'UID' : 'first',
    'trip_creation_time' : 'first',
    'route_schedule_uuid' : 'first',
    'route_type' : 'first',
    'trip_uuid' : 'first',
    'source_center' : 'first',
    'source_name' : 'first',
    
    'destination_center' : 'last',
    'destination_name' : 'last',
    
    'od_start_time' : 'first',
    'od_end_time' : 'first',
    'start_scan_to_end_scan' : 'first',
    
    'actual_distance_to_destination' : 'last',
    'actual_time' : 'last',
    
    'osrm_time' : 'last',
    'osrm_distance' : 'last',
    
    'segment_actual_time_sum' : 'last',
    'segment_osrm_distance_sum' : 'last',
    'segment_osrm_time_sum' : 'last',
    
}

* Grouping mini-trips, sorting by time

In [12]:
segment = df.groupby('segment_key').agg(create_segment_dict, axis=1).reset_index()
#segment = segment.sort_values(by=['segment_key','od_end_time'], ascending=True).reset_index()

In [392]:
segment

,segment_key,UID,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,od_end_time,start_scan_to_end_scan,actual_distance_to_destination,actual_time,osrm_time,osrm_distance,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum
0,trip-153671041653548748IND209304AAAIND000000ACB,fmHUKmPNOZ,2018-09-12 00:00:16.535741,thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...,FTL,trip-153671041653548748,IND209304AAA,Kanpur_Central_H_6 (Uttar Pradesh),IND000000ACB,Gurgaon_Bilaspur_HB (Haryana),2018-09-12 16:39:46.858469,2018-09-13 13:40:23.123744,1339.425944,396.069498,590.0,460.304473,526.839339,419.088245,260.523931,223.487055
1,trip-153671041653548748IND462022AAAIND209304AAA,WGasVV99yU,2018-09-12 00:00:16.535741,thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...,FTL,trip-153671041653548748,IND462022AAA,Bhopal_Trnsport_H (Madhya Pradesh),IND209304AAA,Kanpur_Central_H_6 (Uttar Pradesh),2018-09-12 00:00:16.535741,2018-09-12 16:39:46.858469,1176.195229,292.711155,394.0,765.820455,1041.880174,598.013653,454.293866,331.146670
2,trip-153671042288605164IND561203AABIND562101AAA,IMjLe2P7zY,2018-09-12 00:00:22.886430,thanos::sroute:3a1b0ab2-bb0b-4c53-8c59-eb2a2c0...,Carting,trip-153671042288605164,IND561203AAB,Doddablpur_ChikaDPP_D (Karnataka),IND562101AAA,Chikblapur_ShntiSgr_D (Karnataka),2018-09-12 02:03:09.655591,2018-09-12 03:01:59.598855,107.682373,38.420009,31.0,559.888261,439.847212,41.539905,23.878870,28.513077
3,trip-153671042288605164IND572101AAAIND561203AAB,YkSIUaWh1B,2018-09-12 00:00:22.886430,thanos::sroute:3a1b0ab2-bb0b-4c53-8c59-eb2a2c0...,Carting,trip-153671042288605164,IND572101AAA,Tumkur_Veersagr_I (Karnataka),IND561203AAB,Doddablpur_ChikaDPP_D (Karnataka),2018-09-12 00:00:22.886430,2018-09-12 02:03:09.655591,401.747038,114.121915,40.0,134.253690,234.854753,45.563022,22.856172,16.441720
4,trip-153671043369099517IND000000ACBIND160002AAC,FhCcZkCgdD,2018-09-12 00:00:33.691250,thanos::sroute:de5e208e-7641-45e6-8100-4d9fb1e...,FTL,trip-153671043369099517,IND000000ACB,Gurgaon_Bilaspur_HB (Haryana),IND160002AAC,Chandigarh_Mehmdpur_H (Punjab),2018-09-14 03:40:17.106733,2018-09-14 17:34:55.442454,1089.069442,120.837566,36.0,676.601598,686.957432,599.917973,267.909215,197.082181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22033,trip-153861106442901555IND209304AAAIND208006AAA,xZOgk8zPMa,2018-10-03 23:57:44.429324,thanos::sroute:5609c268-e436-4e0a-8180-3db4a74...,Carting,trip-153861106442901555,IND209304AAA,Kanpur_Central_H_6 (Uttar Pradesh),IND208006AAA,Kanpur_GovndNgr_DC (Uttar Pradesh),2018-10-03 23:57:44.429324,2018-10-04 02:51:27.075797,213.008682,70.426527,92.0,NaN,977.117609,24.397769,7.487007,6.176244
22034,trip-153861115439069069IND627005AAAIND628801AAA,ezINYMou0j,2018-10-03 23:59:14.390954,thanos::sroute:c5f2ba2c-8486-4940-8af6-d1d2a6a...,Carting,trip-153861115439069069,IND627005AAA,Tirunelveli_VdkkuSrt_I (Tamil Nadu),IND628801AAA,Eral_Busstand_D (Tamil Nadu),2018-10-03 23:59:14.390954,2018-10-04 01:44:53.808000,223.987534,84.831708,42.0,86.691781,255.541925,37.892830,42.245413,23.213807
22035,trip-153861115439069069IND627657AAAIND628613AAA,dhg4aPSW4q,None,thanos::sroute:c5f2ba2c-8486-4940-8af6-d1d2a6a...,Carting,trip-153861115439069069,IND627657AAA,Thisayanvilai_UdnkdiRD_D (Tamil Nadu),IND628613AAA,Peikulam_SriVnktpm_D (Tamil Nadu),2018-10-04 03:31:11.183797,2018-10-04 04:16:39.894872,127.822086,58.844795,33.0,771.148885,579.852902,34.219992,10.743811,14.534416
22036,trip-153861115439069069IND628204AAAIND627657AAA,zs2NdBNg0Q,2018-10-03 23:59:14.390954,thanos::sroute:c5f2ba2c-8486-4940-8af6-d1d2a6a...,Carting,trip-153861115439069069,IND628204AAA,Tirchchndr_Shnmgprm_D (Tamil Nadu),IND627657AAA,Thisayanvilai_UdnkdiRD_D (Tamil Nadu),2018-10-04 02:29:04.272194,2018-10-04 03:31:11.183797,154.353713,108.912000,39.0,108.504173,1023.894131,52.272511,36.672717,40.352580


In [343]:
segment['actual_time']

0        590.0
1        394.0
2         31.0
3         24.0
4         36.0
         ...  
16630     21.0
16631     14.0
16632     42.0
16633      9.0
16634     45.0
Name: actual_time, Length: 16635, dtype: float64

In [344]:
segment[segment['trip_uuid'] == 'trip-153741093647649320' ]

,index,segment_key,UID,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,...,od_start_time,od_end_time,start_scan_to_end_scan,actual_distance_to_destination,actual_time,osrm_time,osrm_distance,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum
6642,6642,trip-153741093647649320IND388121AAAIND388620AAB,W2IstUkCZx,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388620AAB,...,2018-09-20 03:21:32.418600,2018-09-20 04:47:45.236797,161.077344,136.700097,62.0,476.825217,785.627241,43.020263,29.456639,32.285165
6643,6643,trip-153741093647649320IND388620AABIND388320AAA,eySzsMscrD,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388620AAB,Khambhat_MotvdDPP_D (Gujarat),IND388320AAA,...,2018-09-20 04:47:45.236797,2018-09-20 06:36:55.627764,227.643735,154.521712,65.0,452.153655,950.096215,22.476446,17.043089,14.358356


In [345]:
segment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16635 entries, 0 to 16634
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   index                           16635 non-null  int64         
 1   segment_key                     16635 non-null  object        
 2   UID                             16635 non-null  object        
 3   trip_creation_time              16635 non-null  object        
 4   route_schedule_uuid             16635 non-null  object        
 5   route_type                      16635 non-null  object        
 6   trip_uuid                       16635 non-null  object        
 7   source_center                   16635 non-null  object        
 8   source_name                     16635 non-null  object        
 9   destination_center              16635 non-null  object        
 10  destination_name                16635 non-null  object        
 11  od

### Calculate time taken between od_start_time and od_end_time and keep it as a feature.

* od_time_diff_hour is matching with start_scan_to_end_time

In [346]:
segment['od_time_diff_hour'] = (segment['od_end_time'] - segment['od_start_time']).dt.total_seconds() /(60)
segment['od_time_diff_hour']

0        1260.604421
1         999.505379
2          58.832388
3         122.779486
4         834.638929
            ...     
16630      60.590521
16631     248.409092
16632     105.656951
16633      62.115193
16634     287.474007
Name: od_time_diff_hour, Length: 16635, dtype: float64

In [347]:
segment['actual_time']

0        590.0
1        394.0
2         31.0
3         24.0
4         36.0
         ...  
16630     21.0
16631     14.0
16632     42.0
16633      9.0
16634     45.0
Name: actual_time, Length: 16635, dtype: float64

In [348]:
create_trip_dict = {
    
    'UID' : 'first',
    'trip_creation_time' : 'first',
    'route_schedule_uuid' : 'first',
    'route_type' : 'first',
    'trip_uuid' : 'first',
    
    'source_center' : 'first',
    'source_name' : 'first',
    
    'destination_center' : 'last',
    'destination_name' : 'last',
    
    'start_scan_to_end_scan' : 'sum',
    'od_time_diff_hour' : 'sum', 
    
    'actual_distance_to_destination' : 'sum',
    'actual_time' : 'sum',
    'osrm_time' : 'sum',
    'osrm_distance' : 'sum',
    
    'segment_actual_time_sum' : 'sum',
    'segment_osrm_distance_sum' : 'sum',
    'segment_osrm_time_sum' : 'sum',
    
}

In [349]:
segment['actual_time']

0        590.0
1        394.0
2         31.0
3         24.0
4         36.0
         ...  
16630     21.0
16631     14.0
16632     42.0
16633      9.0
16634     45.0
Name: actual_time, Length: 16635, dtype: float64

In [350]:
num_cols = ['start_scan_to_end_scan','actual_distance_to_destination','osrm_time',
            'osrm_distance','segment_actual_time_sum','segment_osrm_distance_sum',
           'segment_osrm_time_sum', 'od_time_diff_hour']

In [351]:
trip = segment.groupby('trip_uuid').agg(create_trip_dict).reset_index(drop = True)

In [352]:
trip['actual_time']


0        984.0
1         55.0
2        875.0
3         23.0
4         42.0
         ...  
11298     73.0
11299     21.0
11300     14.0
11301     51.0
11302     45.0
Name: actual_time, Length: 11303, dtype: float64

In [353]:
trip[['segment_actual_time_sum']]

,segment_actual_time_sum
0,710.458402
1,58.982299
2,1221.273123
3,27.234392
4,42.336566
...,...
11298,68.804295
11299,17.895734
11300,26.958970
11301,34.661009


In [354]:
trip[trip['trip_uuid'] == 'trip-153741093647649320' ]

,UID,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,start_scan_to_end_scan,od_time_diff_hour,actual_distance_to_destination,actual_time,osrm_time,osrm_distance,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum
4529,W2IstUkCZx,2018-09-20 02:35:36.476840,thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...,Carting,trip-153741093647649320,IND388121AAA,Anand_VUNagar_DC (Gujarat),IND388320AAA,Anand_Vaghasi_IP (Gujarat),388.721078,195.386819,291.221809,127.0,928.978873,1735.723456,65.496709,46.499729,46.643522


In [355]:
trip[['actual_distance_to_destination', 'osrm_distance']]

,actual_distance_to_destination,osrm_distance
0,688.780653,1568.719513
1,90.712852,674.701965
2,702.921897,2203.434705
3,25.797577,653.106306
4,75.399339,470.277658
...,...,...
11298,174.636780,1095.659376
11299,42.706216,535.318980
11300,42.720209,663.630173
11301,116.740887,1279.436055


### Hypothesis Testing --> Recommendation to the business

In [356]:
trip['destination_name'] = trip['destination_name'].str.lower() # lowering all columns
trip['source_name'] = trip['source_name']

In [357]:
def place2state(x):
    # transform "gurgaon_bilaspur_hb (haryana)" into "haryana"
    state = x.split('(')[1]
    
    return state[:-1] #removing ')' from ending

def place2city(x):
    #we will remove state
    city = x.split(' (')[0]
    
    city = city.split('_')[0]
    
    # Now daling with edge cases
          
    if city == 'pnq vadgaon sheri dpc': return 'vadgaonsheri'

    # ['PNQ Pashan DPC', 'Bhopal MP Nagar', 'HBR Layout PC',
    #  'PNQ Rahatani DPC', 'Pune Balaji Nagar', 'Mumbai Antop Hill']

    if city in ['pnq pashan dpc','pnq rahatani dpc', 'pune balaji nagar']:
        return 'pune'

    if city == 'hbr layout pc' :
        return 'bengaluru'
    if city == 'bhopal mp nagar':
        return 'bhopal'
    if city == 'mumbai antop hill':
        return 'mumbai'

    return city

def place2city_place(x):

    # we will remove state
    x = x.split('(')[0]

    len_ = len(x.split('_'))

    if len_ >= 3:
        return x.split('_')[1]

    # small cities have same city and place name
    if len_ == 2:
        return x.split('_')[0]

    # now we need to deal with edge cases or imporper name convention

    # if len(x.split('_')) == 2:

    return x.split(' ')[0]
    
def place2code(x):
    # we will remove state
    x = x.split('(')[0]

    if len(x.split('_')) >= 3:
        return x.split('_')[-1]

    return 'none'


In [358]:
trip['destination_state'] = trip['destination_name'].apply(lambda x: place2state(x))
trip['destination_city']  = trip['destination_name'].apply(lambda x: place2city(x))
trip['destination_place'] = trip['destination_name'].apply(lambda x: place2city_place(x))
trip['destination_code']  = trip['destination_name'].apply(lambda x: place2code(x))

In [359]:
trip[['destination_state','destination_city','destination_place','destination_code']]

,destination_state,destination_city,destination_place,destination_code
0,uttar pradesh,kanpur,central,6
1,karnataka,doddablpur,chikadpp,d
2,haryana,gurgaon,bilaspur,hb
3,maharashtra,mumbai,mirard,ip
4,karnataka,hospet,hospet,none
...,...,...,...,...
11298,punjab,chandigarh,mehmdpur,h
11299,haryana,faridabad,blbgarh,dc
11300,uttar pradesh,kanpur,central,6
11301,tamil nadu,thisayanvilai,udnkdird,d


In [360]:
trip['trip_creation_time'] = pd.to_datetime(trip['trip_creation_time'])

trip['trip_year'] = trip['trip_creation_time'].dt.year
trip['trip_month'] = trip['trip_creation_time'].dt.month
trip['trip_hour'] = trip['trip_creation_time'].dt.hour
trip['trip_day'] = trip['trip_creation_time'].dt.day
trip['trip_week'] = trip['trip_creation_time'].dt.isocalendar().week
trip['trip_dayofweek'] = trip['trip_creation_time'].dt.dayofweek

In [361]:
trip[['trip_year','trip_month','trip_hour','trip_day','trip_week','trip_dayofweek']]

,trip_year,trip_month,trip_hour,trip_day,trip_week,trip_dayofweek
0,2018,9,0,12,37,2
1,2018,9,0,12,37,2
2,2018,9,0,12,37,2
3,2018,9,0,12,37,2
4,2018,9,0,12,37,2
...,...,...,...,...,...,...
11298,2018,10,23,3,40,2
11299,2018,10,23,3,40,2
11300,2018,10,23,3,40,2
11301,2018,10,23,3,40,2


In [362]:
trip.head()

,UID,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,start_scan_to_end_scan,...,destination_state,destination_city,destination_place,destination_code,trip_year,trip_month,trip_hour,trip_day,trip_week,trip_dayofweek
0,fmHUKmPNOZ,2018-09-12 00:00:16.535741,thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...,FTL,trip-153671041653548748,IND209304AAA,Kanpur_Central_H_6 (Uttar Pradesh),IND209304AAA,kanpur_central_h_6 (uttar pradesh),2515.621173,...,uttar pradesh,kanpur,central,6,2018,9,0,12,37,2
1,IMjLe2P7zY,2018-09-12 00:00:22.886430,thanos::sroute:3a1b0ab2-bb0b-4c53-8c59-eb2a2c0...,Carting,trip-153671042288605164,IND561203AAB,Doddablpur_ChikaDPP_D (Karnataka),IND561203AAB,doddablpur_chikadpp_d (karnataka),509.429411,...,karnataka,doddablpur,chikadpp,d,2018,9,0,12,37,2
2,FhCcZkCgdD,2018-09-12 00:00:33.691250,thanos::sroute:de5e208e-7641-45e6-8100-4d9fb1e...,FTL,trip-153671043369099517,IND000000ACB,Gurgaon_Bilaspur_HB (Haryana),IND000000ACB,gurgaon_bilaspur_hb (haryana),4388.384341,...,haryana,gurgaon,bilaspur,hb,2018,9,0,12,37,2
3,dMTuqfTyeT,2018-09-12 00:01:00.113710,thanos::sroute:f0176492-a679-4597-8332-bbd1c7f...,Carting,trip-153671046011330457,IND400072AAB,Mumbai Hub (Maharashtra),IND401104AAA,mumbai_mirard_ip (maharashtra),538.177129,...,maharashtra,mumbai,mirard,ip,2018,9,0,12,37,2
4,9f9HG74oNM,2018-09-12 00:02:09.740725,thanos::sroute:d9f07b12-65e0-4f3b-bec8-df06134...,FTL,trip-153671052974046625,IND583101AAA,Bellary_Dc (Karnataka),IND583201AAA,hospet (karnataka),419.752382,...,karnataka,hospet,hospet,none,2018,9,0,12,37,2


In [363]:
num_cols = ['start_scan_to_end_scan','actual_distance_to_destination','actual_time','osrm_time',
            'osrm_distance','segment_actual_time_sum','segment_osrm_distance_sum',
           'segment_osrm_time_sum', 'od_time_diff_hour']

In [364]:
trip[num_cols]['actual_time']

0        984.0
1         55.0
2        875.0
3         23.0
4         42.0
         ...  
11298     73.0
11299     21.0
11300     14.0
11301     51.0
11302     45.0
Name: actual_time, Length: 11303, dtype: float64

## **Find outliers in numericals variable(you might find outliers in almost all the variables), and visualize it using visual analysis**

### **Handle the outliers using IQR method**

## **Handling Categorical Variables**

### **Only two route_type-Do one hot encoding**

### **Normalize/Standarize the numerical features using MinMaxScaler or StandardScaler**

## **Recommendation Examples:**

There is a significant difference between OSRM and actual parameters.

### **There is a need to:**

Revisit information fed to routing engine for trip planning. Check for discrepancies with transporters, if the routing engine is configured for optimum results.

North, South and West Zones comidors have significant traffic of orders. But, we have a smaller presence in Central, Eastern and North-Eastern zone. However it would be difficult to conclude this, by looking at just 2 months data. It is worth investigating and increasing our presence in these regions.

From state point of view, we have heavy traffic in Mahrashtra followed by Karnataka. This is a good indicator that we need to plan for resources on ground in these 2 states on priority. Especially, during festive seasons.
